# Functions


In [1]:
import os
import glob
import pandas as pd
import subprocess
import shutil
from typing import List, Dict
import pydicom

# LOCAL
PATIENT_MAP_FPATH  = '/mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_ksp_umcg/workspace/patient_mapping_excel.xlsx'
SCP_COMMANDS_DIR   = '/mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_ksp_umcg/workspace/scp_commands'
ANON_DICOM_DIR     = '/mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_ksp_umcg/workspace/input/sectra_export_pat0001_pat0010_anon_dicoms'
ANON_KSPACE_DIR    = '/mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_ksp_umcg/workspace/output/anon_kspaces'
WORKSPACE_DIR      = '/mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_ksp_umcg/workspace' 
TRANSFER_LOG_FPATH = os.path.join(WORKSPACE_DIR, 'transfer_log.lst')

# REMOTE
REMOTE_BASE_DIR = "/scratch/p290820/datasets/003_umcg_pst_ksps/pat_data"

# INCLUSION: These are the patients that we want to transfer.
patients_to_process_for_now = [
    '0001',  # pat0001
    '0002',  # pat0002
    '0003',  # pat0003
    '0004',  # pat0004
    '0005',  # pat0005
    '0006',  # pat0006
    '0007',  # pat0007
    '0008',  # pat0008
    '0009',  # pat0009
    '0010']  # pat0010


# Overview
patient_map_df = pd.read_excel(PATIENT_MAP_FPATH, sheet_name='Sheet1')
print(patient_map_df.head())
print(os.getcwd())

   pat_seq_id      anon_id mp_bip   mri_date  \
0           1  ANON2784451     mp 2022-04-07   
1           2  ANON9625394    bip 2022-05-08   
2           3  ANON5046358     mp 2022-05-10   
3           4  ANON9616598     mp 2022-05-19   
4           5  ANON8290811    bip 2022-05-22   

                     exclusion_reason  num_lesions  pirads_lesion1  \
0  after brachytherapie, radiotherapy          1.0             NaN   
1                                 NaN          1.0             2.0   
2                                 NaN          1.0             4.0   
3                                 NaN          1.0             2.0   
4                                 NaN          2.0             2.0   

   pirads_lesion2 command_list_for_scp_created  files_in_order  
0             NaN                          NaN             NaN  
1             NaN                          NaN             NaN  
2             NaN                          yes             NaN  
3             NaN             

# Determine for each patient what files need to be transfered.

In [2]:
def get_kspace_files(row: pd.Series) -> list:
    """Find k-space files for a given patient in a given directory.
    
    Args:
        row (pd.Series): row of patient map dataframe
    Returns:
        list: list of k-space file paths
    """
    pat_seq_id_str     = str(row['pat_seq_id']).zfill(4)
    pat_anon_ksp_dir   = os.path.join(ANON_KSPACE_DIR, f"{pat_seq_id_str}_patient_umcg_done")
    ksp_pattern        = os.path.join(pat_anon_ksp_dir, '*T2_TSE_tra*.mrd')
    found_kspace_files = glob.glob(ksp_pattern)
    # make this case insensitive
    ksp_pattern        = os.path.join(pat_anon_ksp_dir, '*t2_tse_tra*.mrd')
    found_kspace_files.extend(glob.glob(ksp_pattern))
    
    num_kspace_files = len(found_kspace_files)

    print(f"\t{num_kspace_files} K-space files candidates:")
    for fpath in found_kspace_files:
        print(f"\t\t{fpath}")
    
    assert num_kspace_files == 2, f"Found {num_kspace_files} k-space files instead of 2"

    return found_kspace_files


def is_relevant_sequence(seq_name: str) -> bool:
    """Filter out sequences that are not T2w"""
    filters = ['fldyn', 'ep_b', 'ep_calc', 'dcadroi']

    for filter in filters:
        if filter.lower() in seq_name.lower():
            return False
        
    return True


def get_t2w_tra_dicom_dir(row: pd.Series) -> str:
    '''
    Get the DICOM directory for the T2W TRA sequence for a given patient.
    Args
        row (pd.Series): row of patient map dataframe
    Returns
        str: DICOM directory for the T2W TRA sequence
    '''
    
    # Get all MRI studies for this patient
    patient_anon_dcm_dir = os.path.join(ANON_DICOM_DIR, row['anon_id'])
    all_mri_studies      = os.listdir(patient_anon_dcm_dir)

    # If there is a study that we want to skip, then remove it from the list
    all_mri_studies = [study for study in all_mri_studies if 'skip' not in study]
    
    for study in all_mri_studies:
        print(f"\t\tMRI study: {study}")

    assert len(all_mri_studies) == 1, f"Found {len(all_mri_studies)} instead of 1, for patient {row['pat_seq_id']}, in {patient_anon_dcm_dir}"

    dcm_dirs_t2_tra = []
    for study_idx, study_name in enumerate(all_mri_studies):
        print(f"\t{len(all_mri_studies)} MRI studies {study_name}   MRI_idx: {study_idx}")

        full_study_dir = os.path.join(patient_anon_dcm_dir, study_name)
        all_sequence_dirs = os.listdir(full_study_dir)

        # Filter out irrelevant sequences
        relevant_sequence_dirs = [seq for seq in all_sequence_dirs if is_relevant_sequence(seq)]

        # filter out each sequence dir that is not named 'archive'
        relevant_sequence_dirs = [seq for seq in relevant_sequence_dirs if 'archive' not in seq.lower()]

        for seq_name in relevant_sequence_dirs:
            full_sequence_dir = os.path.join(full_study_dir, seq_name)
            dicom_files = os.listdir(full_sequence_dir)
            
            assert len(dicom_files) > 15, f"Found less than 15 dicom slices in {full_sequence_dir}"

            # Check the first DICOM file to see if the sequence is T2W TRA
            first_dcm_file_path = os.path.join(full_sequence_dir, dicom_files[0])
            first_dcm_file = pydicom.dcmread(first_dcm_file_path)

            pr_name = first_dcm_file.ProtocolName
            if ('T2' in pr_name and 'tra' in pr_name) or ('t2' in pr_name and 'tra' in pr_name):
                print(f"\t\tT2w tra dir: {full_sequence_dir}")
                dcm_dirs_t2_tra.append(full_sequence_dir)

    assert len(dcm_dirs_t2_tra) == 1, f"Did not find exactly 1 T2W TRA sequence directory for patient {row['pat_seq_id']}, but found {len(dcm_dirs_t2_tra)} instead."

    return dcm_dirs_t2_tra


def get_roi_files(row: pd.Series, study_date: str) -> list:

    # print the number of lesions and their PIRADS scores
    print(f"\t{row['num_lesions']} lesions with PIRADS scores:")
    for lesion_num in range(int(row['num_lesions'])):
        respective_pirads = row[f'pirads_lesion{lesion_num + 1}']       # pirads_lesion1 starts at 1
        print(f"\t\tLesion {lesion_num + 1}: PIRADS {respective_pirads}")

    nifti_dir = os.path.join(WORKSPACE_DIR, 'input', 'sectra_export_pat0001_pat0010_niftis', row['anon_id'], study_date)

    roi_pattern = os.path.join(nifti_dir, '*ROI*.mha')
    roi_files   = glob.glob(roi_pattern)
    roi_pattern = os.path.join(nifti_dir, '*roi*.mha')
    roi_files.extend(glob.glob(roi_pattern))

    # if we did not find any ROI files, then try to find Nifti files instead
    if len(roi_files) == 0:
        roi_pattern = os.path.join(nifti_dir, '*ROI*.nii.gz')
        roi_files   = glob.glob(roi_pattern)
        roi_pattern = os.path.join(nifti_dir, '*roi*.nii.gz')
        roi_files.extend(glob.glob(roi_pattern))

    # count the number of ROIs that we expect to find
    n_rois_to_be_found = 0
    for lesion_num in range(int(row['num_lesions'])):
        respective_pirads = row[f'pirads_lesion{lesion_num + 1}']       # pirads_lesion1 starts at 1
        if respective_pirads == 2:
            print(f"\tFOUND PIRADS 2 LESION, MUST BE SEGMENTED MANUALLY!!!!!!!!!!!!!!")
        if respective_pirads > 2:                                   # only count lesions with PIRADS > 2, because we only have ROIs for those
            n_rois_to_be_found += 1                                 # count the number of ROIs that we expect to find

    print(f"\t{n_rois_to_be_found} ROI files candidates:")
    for idx, roi_file in enumerate(roi_files):
        print(f"\t\t{idx+1} ROI file: {roi_file}")

    # print that if we have the right number of ROIs
    if len(roi_files) == n_rois_to_be_found:
        print(f"\t\tFound {len(roi_files)} ROI files")
    
    # assert that we found the correct number of ROIs
    assert len(roi_files) == n_rois_to_be_found, f"Found {len(roi_files)} ROI files instead of {n_rois_to_be_found}"

    return roi_files
    

def extract_dicom_study_date_from_dicom_dir(dcm_dir: str, verbose=False) -> str:
    """Extract the DICOM study date from the DICOM directory name."""
    # Example: 		T2w tra dir: /mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_ksp_umcg/workspace/input/sectra_export_pat0001_pat0010_anon_dicoms/ANON2784451/2022-04-07/tse2d1_25_1.3.12.2.1107.5.2.19.46133.2022040713122344044291325.0.0.0
    
    # get the DICOM study date
    study_date = dcm_dir.split('/')[-2]
    
    if verbose:
        print(f"\t\t\tExtract Study date: {study_date}")

    return study_date


def find_t2w_tra_nifti(row: pd.Series, study_date: str, dcm_dir: str, verbose=False) -> list:
    nifti_dir = os.path.join(WORKSPACE_DIR, 'input', 'sectra_export_pat0001_pat0010_niftis', row['anon_id'], study_date)

    # get the DICOM directory name
    dcm_dir_name = dcm_dir.split('/')[-1]
    
    # find the Nifti files that match the DICOM directory name
    nifti_pattern = os.path.join(nifti_dir, f'*{dcm_dir_name}*.nii.gz')
    nifti_files   = glob.glob(nifti_pattern)

    if verbose:
        for idx, nifti_file in enumerate(nifti_files):
            print(f"\t{idx+1} Nifti file: {nifti_file}")

    assert len(nifti_files) == 1, f"Did not find exactly 1 T2W TRA Nifti file for patient {row['anon_id']}, but found {len(nifti_files)} instead."

    return nifti_files


####################################################################################################

files_to_transfer = []
for index, row in patient_map_df.iterrows():

    seq_id = str(row['pat_seq_id']).zfill(4)

    # Break when we reach the end of the patient map or if the seq_id is not in the patients_to_process_for_now list 
    if pd.isna(row['anon_id']) or seq_id not in patients_to_process_for_now:
        break

    print(f"\t\nPatient {seq_id} with anon ID {row['anon_id']}")

    # Skip patients that have an exclusion reason
    if not pd.isnull(row['exclusion_reason']):
        print(f"Exclusion reason: {row['exclusion_reason']}")
        continue

    # Get the k-space files, DICOM directory, ROI files and Nifti files
    ksp_files   = get_kspace_files(row)
    dcm_dirs    = get_t2w_tra_dicom_dir(row)
    study_date  = extract_dicom_study_date_from_dicom_dir(dcm_dirs[0], verbose=False)
    roi_files   = get_roi_files(row, study_date)
    nifti_files = find_t2w_tra_nifti(row, study_date, dcm_dirs[0], verbose=True)

    has_passed = True
    if len(ksp_files) != 2:
        print(f"FAILED: Found {len(ksp_files)} k-space files instead of 2")
        has_passed = False
    if len(dcm_dirs) != 1:
        print(f"FAILED: Found {len(dcm_dirs)} DICOM directories instead of 1")
        has_passed = False
    if len(roi_files) != row['num_lesions']:
        print(f"FAILED: Found {len(roi_files)} ROI files instead of {row['num_lesions']}")
        has_passed = False
    if len(nifti_files) != 1:
        print(f"FAILED: Found {len(nifti_files)} Nifti files instead of 1")
        has_passed = False
    if has_passed:
        print(f"PASSED: ALL TESTS!!!!")
    
    patient_info_dict = {
        'pat_seq_id' : row['pat_seq_id'],
        'anon_id'    : row['anon_id'],
        'study_date' : study_date,
        'ksp_files'  : ksp_files,
        'dcm_dirs'   : dcm_dirs,
        'roi_files'  : roi_files,
        'nifti_files': nifti_files
    }
    files_to_transfer.append(patient_info_dict)

	
Patient 0001 with anon ID ANON2784451
Exclusion reason: after brachytherapie, radiotherapy
	
Patient 0002 with anon ID ANON9625394
	2 K-space files candidates:
		/mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_ksp_umcg/workspace/output/anon_kspaces/0002_patient_umcg_done/meas_MID00039_FID701346_T2_TSE_tra_obl-out_1.mrd
		/mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_ksp_umcg/workspace/output/anon_kspaces/0002_patient_umcg_done/meas_MID00039_FID701346_T2_TSE_tra_obl-out_2.mrd
		MRI study: 2022-05-08
	1 MRI studies 2022-05-08   MRI_idx: 0
		T2w tra dir: /mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_ksp_umcg/workspace/input/sectra_export_pat0001_pat0010_anon_dicoms/ANON9625394/2022-05-08/tse2d1_25_1.3.12.2.1107.5.2.19.46133.2022050807571572157809275.0.0.0
	1.0 lesions with PIRADS scores:
		Lesion 1: PIRADS 2.0
	FOUND PIRADS 2 LESION, MUST BE SEGMENTED MANUALLY!!!!!!!!!!!!!!
	0 ROI files candidates:
		Found 0 ROI files
	1 Nifti file: /mnt/c/Users/qvloh/Documents/

# SCP Program - Transfer to Habrok Directories

In [3]:
def get_all_files_in_dir(dir: str) -> list:
    """Get all files in a directory."""
    all_files = []
    for root, dirs, files in os.walk(dir):
        for file in files:
            all_files.append(os.path.join(root, file))
    return all_files


def execute_scp(local_path, remote_path, retries=3):
    '''
    Description:
        Execute an scp command. If the command fails, then retry a number of times.
        Log the command in a log file if it succeeds.
    Args:
        local_path (str): local file path
        remote_path (str): remote file path 
        retries (int): number of retries if the command fails (default: 3)
    '''

    # Check for scp availability
    if shutil.which("scp") is None:
        print("scp command not found.")
        return False

    # Check the log file to see if this file has already been transferred
    try:
        with open(TRANSFER_LOG_FPATH, 'r') as f:
            log_entries = f.read()
        if f"{local_path} --> {remote_path}\n" in log_entries:
            print(f"File {local_path} has already been transferred to {remote_path}. Skipping...")
            return True
    except FileNotFoundError:
        print(f"Log file {TRANSFER_LOG_FPATH} not found. Creating a new log file.")

    # Build the scp command
    command = f"scp {local_path} p290820@interactive1.hb.hpc.rug.nl:{remote_path}"
    
    for attempt in range(retries):
        try:
            result = subprocess.run(command, shell=True, text=True, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
            
            if result.returncode == 0:
                print(f"Successfully executed command: {command}")

                # Write to the log file
                with open(TRANSFER_LOG_FPATH, 'a') as f:
                    f.write(f"{local_path} --> {remote_path}\n")

                return True
            else:
                print(f"Failed to execute command: {command}. Error: {result.stderr}. Retrying...")

        except Exception as e:
            print(f"Exception while executing command: {command}. Exception: {str(e)}. Retrying...")

    print(f"Failed to execute command after {retries} retries: {command}")
    return False
    

def transfer_files(files_to_transfer: List[Dict], remote_base_dir: str):
    '''
    Description:
        Transfer the files to the remote server.
    Args:
        files_to_transfer (List[Dict]): list of dictionaries containing the files to transfer
        remote_base_dir (str): base directory on the remote server
    '''

    for patient_info_dict in files_to_transfer:

        seq_id = str(patient_info_dict['pat_seq_id']).zfill(4)
        remote_pat_dir = os.path.join(remote_base_dir, f"{seq_id}_{patient_info_dict['anon_id']}")

        print(f"Pat{seq_id}, {patient_info_dict['anon_id']} and study date: {patient_info_dict['study_date']}")
        
        # 1: Transfer the kspace files
        print(f"\tkspace to transfer: {len(patient_info_dict['ksp_files'])}")
        for fpath_ksp in patient_info_dict['ksp_files']:
            print(f"\t\t{fpath_ksp}")
            execute_scp(fpath_ksp, remote_pat_dir)

        # 2: Transfer the DICOM files
        print(f"\tDICOM dirs to transfer: {len(patient_info_dict['dcm_dirs'])}")
        for dcm_dir in patient_info_dict['dcm_dirs']:
            print(f"\t\t{dcm_dir}")
            
            # get all files in the DICOM directory
            dcm_fpaths = get_all_files_in_dir(dcm_dir)
            for dcm_fpath in dcm_fpaths:
                print(f"\t\t\t{dcm_fpath}")
                # all of these files should be transferred to the same remote directory, in the patient t2w_tra_dicom directory
                remote_dcm_dir = os.path.join(remote_pat_dir, 't2w_tra_dicom')
                execute_scp(dcm_fpath, remote_dcm_dir)

        # 3: Transfer the ROI files
        print(f"\tROI files to transfer: {len(patient_info_dict['roi_files'])}")
        for fpath_roi in patient_info_dict['roi_files']:
            print(f"\t\t{fpath_roi}")
            execute_scp(fpath_roi, remote_pat_dir)

        # 4: Transfer the Nifti files
        print(f"\tNifti files to transfer: {len(patient_info_dict['nifti_files'])}")
        for fpath_nifti in patient_info_dict['nifti_files']:
            print(f"\t\t{fpath_nifti}")
            execute_scp(fpath_nifti, remote_pat_dir)
        
        print('\n')


###### Transfer the files
transfer_files(files_to_transfer, remote_base_dir=REMOTE_BASE_DIR)



Pat0002, ANON9625394 and study date: 2022-05-08
	kspace to transfer: 2
		/mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_ksp_umcg/workspace/output/anon_kspaces/0002_patient_umcg_done/meas_MID00039_FID701346_T2_TSE_tra_obl-out_1.mrd
File /mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_ksp_umcg/workspace/output/anon_kspaces/0002_patient_umcg_done/meas_MID00039_FID701346_T2_TSE_tra_obl-out_1.mrd has already been transferred to /scratch/p290820/datasets/003_umcg_pst_ksps/pat_data/0002_ANON9625394. Skipping...
		/mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_ksp_umcg/workspace/output/anon_kspaces/0002_patient_umcg_done/meas_MID00039_FID701346_T2_TSE_tra_obl-out_2.mrd
Successfully executed command: scp /mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_ksp_umcg/workspace/output/anon_kspaces/0002_patient_umcg_done/meas_MID00039_FID701346_T2_TSE_tra_obl-out_2.mrd p290820@interactive1.hb.hpc.rug.nl:/scratch/p290820/datasets/003_umcg_pst_ksps/pat_data/0002_ANON9625394
	